In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import eegraph
import stellargraph as sg
from stellargraph import StellarGraph
import os



## Variables a usar

In [ ]:
path = '../GCNN/Scripts/data/data_vigilia'              #<--------------- PATH TO FOLDER CONTAINING EEGs
window_size_class_0 = 1                                 #<--------------- CLASS 0 WINDOW SIZE
window_size_class_1 = 1                               #<--------------- CLASS 1 WINDOW SIZE
connectivity_number_total = 1                                
exclude = [] 
#CONNECTIVITY MEASURES USED
connectivity= "squared_coherence"
bands = ['theta', 'alpha', 'beta']

## Funcion para convertir eegraf a StellarGraph

In [ ]:
def convertir_eegraph(G):
    array_grafos= []
    #recorremos los grafos
    for i in range(len(G)):
        #convertimos el grafo
        s = StellarGraph.from_networkx(G[i])
        #los guardamos en un array
        array_grafos.append(s)
    return array_grafos


## funcion modelar Grafo

In [ ]:
def modelate_with_different_connectivity(window_size, G, connectivity_number_total, connectivity, bands):
    array_grafos = []
    #recorremos el numero de conectividades elegidas
    for i in range(connectivity_number_total):
        #creamos nuestro grafo modelado y guardamos nuestra matriz de adyacencia(no nos hace falta pero son datos que podrían venir bien en un futuro)
        graphs, c_m = G.modelate(window_size = window_size, connectivity = connectivity, bands = bands) #<--------------- THRESHOLD , threshold = 0.2          
        #llamamos recursivamente a la función para convertir los grafos de tipo networkx en un objeto manejable por Stellar graph
        array_grafos = array_grafos + convertir_eegraph(graphs)
    return  array_grafos

## Abrir directorios e importar datos

In [ ]:
def open_data_directories(path, window_size_class_0, window_size_class_1, exclude, connectivity_number_total,connectivity,bands):
    array_grafos = []
    class_files = os.listdir(path)
    #recorremos todas las carpetas y ficheros del path dado
    for entry in class_files:
        eeg_files = os.listdir(path + '/' + entry)
        for eeg in eeg_files:
            eeg_path = (path + '/' + entry + '/' + eeg)
            print(eeg_path, entry)#imprimimos el path del archivo completo
            G = eegraph.Graph() #creamos un objeto de tipo eegraph
            G.load_data(path= eeg_path, exclude = exclude) # llamamos al objeto y cargamos el archivo seleccionado
            
            if(entry == '1'):
                window_size = window_size_class_1
            elif (entry == '0'):
                window_size = window_size_class_0
            #llamamos recursivamente a la función para modelar nuestro grafo con las medidas deseadas
            array_grafos = array_grafos + modelate_with_different_connectivity(window_size , G, connectivity_number_total,connectivity,bands)
        
    return array_grafos

## llamada a las funciones

In [ ]:
graphs = open_data_directories(path, window_size_class_0, window_size_class_1, exclude, connectivity_number_total,connectivity,bands)
